Resolver o dataset iris com tecnicas de Redes Neurais.

Multi-layer Perceptron (MLP)

In [1]:
from sklearn.neural_network import MLPClassifier

from sklearn import datasets # para utilizar o iris dataset
iris = datasets.load_iris()

from sklearn.decomposition import PCA # utilizaremos para fazer a redução de dimensionalidade 

In [2]:
print(iris.DESCR)

Iris Plants Database

Notes
-----
Data Set Characteristics:
    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20  0.76     0.9565  (high!)
    ============== ==== ==== ======= ===== ====================

    :Missing Attribute Values: None
  

In [3]:
X = iris.data
Y = iris.target

print(X.shape)
print(Y.shape)

(150L, 4L)
(150L,)


In [4]:
# Aplicando a tecninca de redução de dimensionalidade

X_reduced = PCA(n_components=3).fit_transform(X)

In [5]:
# Separando os dados de treino e teste

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_reduced, Y, test_size=0.33, random_state=0)

In [6]:
# Realizando o treinamento

clf = MLPClassifier(
    hidden_layer_sizes=250, 
    activation='relu', 
    solver='adam', 
    alpha=0.001, 
    learning_rate_init=0.001, 
    max_iter=400,
    random_state=0)


clf.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=250, learning_rate='constant',
       learning_rate_init=0.001, max_iter=400, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=0, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [7]:
# Realizando as predições.

predicoes = clf.predict(X_test)

In [8]:
# Gerando a acuracia do modelo.

from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, predicoes)
print('Accuracy score: {0:.1f}%.'.format(accuracy * 100))

Accuracy score: 98.0%.


In [9]:
from sklearn.model_selection import KFold

def treino_do_classificador(clf, X, y):
    clf.fit(X,y)

def predicao(clf, X_test, y_test):
    predicao = clf.predict(X_test)
    return accuracy_score(y_test, predicao)

def treino_e_predicoes(clf, X_train, y_train, X_test, y_test):
    k_fold = KFold(3)
    for k, (train, test) in enumerate(k_fold.split(X_train, y_train)):
        clf.fit(X_train[train], y_train[train])
        print("k-fold %s , score: %.4f, acurracy: %.4f" % (k, clf.score(X_train[test], y_train[test]), predicao(clf, X_train, y_train)))    
    
    treino_do_classificador(clf, X_train, y_train)
    
    print "Acurracy para o conjunto de treino: {:.2f}%.".format(predicao(clf, X_train, y_train))
    print "Acurracy para o conjunto de teste: {:.2f}%.".format(predicao(clf, X_test, y_test))

In [10]:
treino_e_predicoes(clf, X_train, y_train, X_test, y_test)

k-fold 0 , score: 0.9412, acurracy: 0.9700


C:\Users\dfabiano\AppData\Local\Continuum\anaconda2\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


k-fold 1 , score: 1.0000, acurracy: 0.9900
k-fold 2 , score: 0.9091, acurracy: 0.9700
Acurracy para o conjunto de treino: 0.99%.
Acurracy para o conjunto de teste: 0.98%.


In [11]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, X_test, y_test, cv=5)
print("Accuracy: %0.4f" % (scores.mean()))

Accuracy: 0.9378


In [13]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {
        'activation' : ['identity', 'logistic', 'tanh', 'relu'],
        'solver' : ['lbfgs', 'sgd', 'adam'],
        'hidden_layer_sizes': [
             (1,),(2,),(3,),(4,),(5,),(6,),(7,),(8,),(9,),(10,),(11,),
             (12,),(13,),(14,),(15,),(16,),(17,),(18,),(19,),(20,),(21,)
         ]
    }
]

clf_rede_neural = MLPClassifier()
clf = GridSearchCV(clf_rede_neural, param_grid, cv=3, scoring='accuracy')
clf.fit(X_train,y_train)
print(clf.best_params_)

clf_best = clf.best_estimator_
clf_best.fit(X_train, y_train)

predicoes_best = clf_best.predict(X_test)

accuracy = accuracy_score(y_test, predicoes_best)
print('Accuracy score best : {0:.1f}%.'.format(accuracy * 100))

{'activation': 'identity', 'solver': 'lbfgs', 'hidden_layer_sizes': (1,)}
Accuracy score best : 96.0%.
